In [160]:
import json
import pandas as pd

In [161]:
# Load extracted data
with open('../../outputs/nv_task/extractions/full_lb_nv_taskstructured-zeroshot_gpt-4o-mini-2024-07-18.json') as f:
    nv_task_gpt_json = json.load(f)

with open('../../outputs/nv_task/extractions/full_lb_nv_taskstructured-zeroshot_gpt-4o-2024-08-06.json') as f:
    nv_task_gpt4_json = json.load(f)

# Convert to dictionary and clean
def _clean_extracted_data(data):
    data_dict = {}
    for item in data:
        pmcid = item['pmcid']
        data_dict[pmcid] = {}
        for key, value in item.items():
            value = None if value in ['null', []] else value
            if key == 'Modality' and value:
                value = [v.replace(' ', '') for v in value if v]

            data_dict[pmcid][key] = value

        for bad_key in ['Fmritasks', 'Behavioraltasks']:
            if bad_key in data_dict[pmcid]:
                data_dict[pmcid].pop(bad_key)

        fMRITasks = data_dict[pmcid].get('fMRITasks') or []

        for k in ['TaskName', 'TaskDescription']:
            data_dict[pmcid][k] = []
            for task in fMRITasks:
                if not task['RestingState']:
                    data_dict[pmcid][k].append(task.get(k, None))

        has_resting_state = any(task['RestingState'] for task in fMRITasks)
        data_dict[pmcid]['HasRestingState'] = has_resting_state

        
            
    return data_dict
nv_task_gpt = _clean_extracted_data(nv_task_gpt_json)
nv_task_gpt4 = _clean_extracted_data(nv_task_gpt4_json)

In [162]:
# Load annotations

from labelrepo.projects.nv_task import load_annotations

annotations = load_annotations()
annotations = annotations[annotations['annotator_name'] != 'alice_chen']

# Process annotations
def _get_task_name(rows):
    # Add TaskName, replacing 'None' and 'Unsure' with 'n/a'
    rows = rows[rows.label_name == 'TaskName']
    task_names = []
    for _, row in rows.iterrows():
        if row['None'] or row['Unsure']:
            task_names.append('n/a')
        else:
            task_names.append(row['selected_text'])
    return task_names

# Convert to comparable dictionary
annotations_summary = {}
for pmcid, df in annotations.groupby('pmcid'):
    HasRestingState = 'DesignType-RestingState' in df.label_name.values

    s = {
        'pmcid': pmcid,
        'HasRestingState': HasRestingState,
        'annotator_name': df.annotator_name.iloc[0],
        'Exclude': next(
            (label.split('-', 1)[1] for label in df.label_name if label.startswith('Exclude')), None
        ),
        'Modality': [
            label.split('-', 1)[1] for label in df.label_name if label.startswith('Modality')
        ] or None,
    }

    df_abstract = df[df.section == 'abstract']
    abstract_tasks = _get_task_name(df_abstract)

    df_body = df[df.section == 'body']
    body_tasks = _get_task_name(df_body)

    # Use body tasks if available, otherwise use abstract tasks
    s['TaskName'] = body_tasks or abstract_tasks

    for k in ['TaskDescription', 'Condition', 'ContrastDefinition']:
        s[k] = df_body.loc[df_body.label_name == k, 'selected_text'].tolist() or None

    annotations_summary[pmcid] = s

### Evaluate

In [163]:
import re
from thefuzz import process, fuzz

def _clean(x):
    _li =  [x.lower().replace('paradigm', '').replace('task', '').replace('‐', '').strip() for x in (x or [])]

    return [re.sub(r'\s*\([^)]*\)', '', x) for x in _li]

def _compare(x, y):
    x = x or ''
    y = y or ''
    return x.replace('-', '').lower() == y.replace('-', '').lower()

def _score_comparison(x, y):
    res = _compare(x, y)
    return 100 if res else 0

def score_combinations(correct_labels, extracted_labels, scorer=fuzz.token_set_ratio):
    correct_labels = _clean(correct_labels)
    extracted_labels = _clean(extracted_labels)
    
    matched_labels = []
    
    while correct_labels and extracted_labels:
        # Collect all matches and their scores
        all_matches = []
        for correct_label in correct_labels:
            matches = process.extract(correct_label, extracted_labels, limit=None, scorer=scorer)
            for matched_label, score in matches:
                all_matches.append((correct_label, matched_label, score))

        # Sort all matches by score in descending order
        all_matches.sort(key=lambda x: x[2], reverse=True)

        # Take the highest match
        best_match = all_matches[0]
        correct_label, matched_label, score = best_match

        # Append to results and remove matched labels
        matched_labels.append(score)
        correct_labels.remove(correct_label)
        extracted_labels.remove(matched_label)

    matched_labels = (sum(matched_labels) / len(matched_labels)) / 100 if matched_labels else pd.NA

    return matched_labels

In [164]:
def run_all_comparisons(annotations_summary, extractions):
    # Compare annotations to extractions
    all_scores = {}

    for pmcid, ann in annotations_summary.items():
        gpt_subset = extractions[pmcid]
        # Compare Modality
        if ann['Modality']:
            modality_score = score_combinations(ann['Modality'], gpt_subset['Modality'], scorer=_score_comparison)
        else:
            modality_score = None

        all_scores[pmcid] = {
            'Exclude': _compare(ann['Exclude'], gpt_subset['Exclude']),
            'Modality': modality_score,
            'HasRestingState': ann['HasRestingState'] == gpt_subset['HasRestingState']
        }
        
        for k in ['TaskName', 'TaskDescription']:
            all_scores[pmcid][k] = score_combinations(ann[k], gpt_subset[k])

    # Calculate scores
    n_articles = len(annotations_summary.keys())

    all_scores_df = pd.DataFrame(all_scores).T

    return all_scores_df

all_scores_df = run_all_comparisons(annotations_summary, nv_task_gpt)
all_scores_df4 = run_all_comparisons(annotations_summary, nv_task_gpt4)


In [165]:
score_combinations([], [])

<NA>

In [166]:
exclude_idx = [_p for _p, v in annotations_summary.items() if v['Exclude'] is not None]
has_task_name = [_p for _p, v in annotations_summary.items() if 'n/a' not in v['TaskName']]
has_task_noname = [_p for _p, v in annotations_summary.items() if 'n/a' in v['TaskName']]

def compute_subset(df):
    # Overall scores
    mean_all = df.mean()
    all_n = df.shape[0]

    # Excluding articles with 'Exclude' label
    no_exclude_res = df.loc[~df.index.isin(exclude_idx)]
    no_exclude_res_n = no_exclude_res.shape[0]


    # For papers with a clearly defined task name
    has_task_name_res = df.loc[df.index.isin(has_task_name)]
    has_task_name_n = has_task_name_res.shape[0]


    # For papers with a task-based design but no annotated task name
    has_task_name_noname = df.loc[df.index.isin(has_task_noname)]
    has_task_name_noname_n = has_task_name_noname.shape[0]

    # Combine results into a single dataframe
    results = pd.concat([mean_all, no_exclude_res.mean(), has_task_name_res.mean(), has_task_name_noname.mean()], axis=1)
    results.columns = ['All', 'No Exclude', 'Has Task Name', 'Has Task with No Name']

    # Change dtype to float
    results = results.astype(float)
    results = results.round(2).T

    combine_ns = pd.Series([all_n, no_exclude_res_n, has_task_name_n, has_task_name_noname_n], index=results.index, name='n').round(0)
    results.insert(loc = 0, column = 'n', value = combine_ns)

    return results

results = compute_subset(all_scores_df)
results4 = compute_subset(all_scores_df4)


In [167]:
results

,n,Exclude,Modality,HasRestingState,TaskName,TaskDescription
All,104,0.94,0.99,0.91,0.78,0.77
No Exclude,95,1.00,0.99,0.91,0.78,0.77
Has Task Name,86,0.93,0.99,0.91,0.94,0.77
Has Task with No Name,18,1.00,1.00,0.94,0.18,0.77


In [168]:
results4

,n,Exclude,Modality,HasRestingState,TaskName,TaskDescription
All,104,1.0,0.99,0.94,0.75,0.69
No Exclude,95,1.0,0.99,0.94,0.75,0.69
Has Task Name,86,1.0,0.99,0.93,0.92,0.69
Has Task with No Name,18,1.0,1.00,1.00,0.18,0.73


### Manual comparison

In [169]:
gpt4mini = pd.DataFrame(nv_task_gpt).T
gpt4 = pd.DataFrame(nv_task_gpt4).T
gpt4mini['annotator_name'] = 'gpt-4o-mini'
gpt4['annotator_name'] = 'gpt-4o'

combined_df = pd.concat([pd.DataFrame(annotations_summary).T, gpt4mini, gpt4], sort=False)

In [177]:
import pandas as pd
from IPython.display import display

pd.set_option('max_colwidth', 300)
    

def _display(df, pmcids=None, exclude_cols=['pmcid', 'fMRITasks', 'BehavioralTasks', 'RestingState', 'RestingStateMetadata', 'Condition', 'ContrastDefinition', 'Restingstate', 'TaskDuration', 'TaskDesign']):
    cols = list(set(df.keys()) - set(exclude_cols))
    if pmcids:
        df = df[df.pmcid.isin(pmcids)]
    for _, _df in df.groupby('pmcid'):
        display(_df[cols])

In [178]:
# Diff on Exclude 

diff_exclude = set(all_scores_df[all_scores_df['Exclude'] == False].index.tolist()).union(set(all_scores_df4[all_scores_df4['Exclude'] == False].index.tolist()))
if diff_exclude:
    _display(combined_df, diff_exclude)

In [179]:
# From now on, we will exclude Excluded articles
combined_df = combined_df[~combined_df.index.isin(exclude_idx)]

In [180]:
# Diff on has_resting_state

diff_res = set(all_scores_df[all_scores_df['HasRestingState'] < 1].index.tolist()).union(set(all_scores_df4[all_scores_df['HasRestingState'] < 1].index.tolist()))

if diff_res:
    _display(combined_df, diff_res)

,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
3825257,delavega_nv,[fMRI-BOLD],[],None,True,None,NaN
3825257,gpt-4o-mini,[fMRI-BOLD],[],None,False,[],"To investigate whether variations in the experience of reading across individuals have a basis in the brain's functional architecture, particularly focusing on the anterior medial prefrontal cortex (aMPFC) and posterior cingulate cortex (PCC) as part of the default mode network (DMN)."
3825257,gpt-4o,[fMRI-BOLD],[],None,False,[],To explore whether the intrinsic functional connectivity of key hubs of the default mode network (DMN) is predictive of individual differences in reading comprehension and task focus.


,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
4110030,delavega_nv,[fMRI-BOLD],[],None,True,None,NaN
4110030,gpt-4o-mini,[fMRI-BOLD],[],None,False,[],To investigate sex differences and menstrual cycle effects in resting state functional connectivity of fronto-parietal cognitive control networks.
4110030,gpt-4o,[fMRI-BOLD],[],None,True,[],Investigation of sex differences and menstrual cycle effects in resting state cognitive control networks using fMRI.


,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
4386762,delavega_nv,[fMRI-BOLD],[],None,True,None,NaN
4386762,gpt-4o-mini,[fMRI-BOLD],[],None,False,[],"To investigate the relationship between ALFF and global FC with a voxel-based approach in the healthy and diseased brain, particularly in patients with Alzheimer's Disease (AD) and Mild Cognitive Impairment (MCI)."
4386762,gpt-4o,[fMRI-BOLD],[],None,True,[],"To investigate the relationship between ALFF and global FC with a voxel-based approach in the healthy and diseased brain, specifically in degenerative dementia."


,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
4440210,delavega-aliceoverlap,[fMRI-BOLD],[emotional matching task],None,True,[Subjects were shown triplets of geometric shapes (as neutral stimuli) and of threatening scenes as well as fearful faces (as emotional conditions) presented in alternating blocks of neutral and emotional stimuli.],NaN
4440210,gpt-4o-mini,[fMRI-BOLD],[Emotional Matching Task],None,False,[Subjects were shown triplets of geometric shapes (neutral stimuli) and of threatening scenes as well as fearful faces (emotional conditions) presented in alternating blocks of neutral and emotional stimuli.],To investigate activations in the amygdala region to the presentation of emotional faces and to understand the origins of signal fluctuations and their heterogeneity in this region.
4440210,gpt-4o,[fMRI-BOLD],[],None,False,[],None


,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
9148994,delavega_nv,[fMRI-BOLD],[spatial contextual memory task],None,True,None,NaN
9148994,gpt-4o-mini,[fMRI-BOLD],[Spatial Contextual Memory Task],None,False,"[Participants performed a spatial contextual memory task consisting of an encoding, consolidation, and retrieval phase.]",This study examined age effects on consolidation-related neural mechanisms and their susceptibility to interference using functional magnetic resonance imaging data from younger and older healthy participants.
9148994,gpt-4o,[fMRI-BOLD],[],None,True,[],This study examined age effects on consolidation-related neural mechanisms and their susceptibility to interference using functional magnetic resonance imaging data from younger and older healthy participants.


,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
9202476,delavega-aliceoverlap,"[fMRI-BOLD, DiffusionMRI]",[],None,True,None,NaN
9202476,gpt-4o-mini,"[fMRI-BOLD, DiffusionMRI]",[match-to-sample (MTS) task],None,False,[This computerized task assesses visual search and attention by presenting a complex figure and requiring subjects to match it with patterns shown in the periphery.],"The aim of the study was to evaluate correlation of sleep quality with visual attention and search, functional, and tracts properties of the DAN and VAN."
9202476,gpt-4o,"[fMRI-BOLD, DiffusionMRI, StructuralMRI]",[],None,True,[],"The aim of the study was to evaluate correlation of sleep quality with visual attention and search, functional, and tracts properties of the DAN and VAN."


,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
9729227,delavega_nv,[fMRI-BOLD],[],None,True,None,NaN
9729227,gpt-4o-mini,[fMRI-BOLD],[],None,False,[],To explore the subcortical anatomy of the ventral and dorsal attention networks by aligning individual resting-state functional maps in a common functional space.
9729227,gpt-4o,[fMRI-BOLD],[],None,False,[],To map the subcortical architecture of the Ventral and Dorsal Attention Networks using advanced methods of functional alignment applied to resting-state fMRI.


,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
10028637,delavega_nv,[fMRI-BOLD],[n/a],None,True,None,NaN
10028637,gpt-4o-mini,[fMRI-BOLD],[Auditory Target Detection Task],None,False,[A computerized auditory target detection task aimed to assess individual responsiveness differences during moderate anaesthesia.],"To investigate whether variability or impairments in functional connectivity within and between the dorsal attention network (DAN), executive control network (ECN), and default mode network (DMN) underlie individual differences in responsiveness during propofol anaesthesia."
10028637,gpt-4o,"[fMRI-BOLD, StructuralMRI]",[Narrative Understanding Task],None,True,[Participants listened to a plot-driven auditory narrative while undergoing fMRI scans to investigate perceptual or high-level attention processes under anaesthesia.],"To investigate whether variability or impairments in functional connectivity (FC) within and between the dorsal attention network (DAN), executive control network (ECN), and default mode network (DMN) underlie individual differences in responsiveness during propofol anaesthesia."


,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
10031743,delavega_nv,"[StructuralMRI, DiffusionMRI, fMRI-CBF]",[],None,True,None,NaN
10031743,gpt-4o-mini,"[StructuralMRI, DiffusionMRI]",[],None,False,[],This study tested whether four FTD dimensions differ in their association with brain perfusion and brain structure.
10031743,gpt-4o,"[StructuralMRI, DiffusionMRI, fMRI-CBF]",[],None,False,[],"This study investigates the neural correlates of four formal thought disorder (FTD) dimensions in patients with schizophrenia spectrum disorders using multimodal brain imaging, including resting-state cerebral blood flow (CBF) and brain structure assessments."


Although with this prompt GPT (especially mini), sometimes fails to detect a resting state scan, at least it rarely mentions it as a task without labeling it as resting-state.

Thus it would be fairly easy to exclude studies with no fMRITasks listed, and if any are resting state they can be ignored. 

In [181]:
diff_dt = [6331309, 7426775, 7582181, 8564184, 9202476]
_display(combined_df, diff_dt)

,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
6331309,delavega_nv,[StructuralMRI],[],None,False,None,NaN
6331309,gpt-4o-mini,[StructuralMRI],[],None,False,[],"This study aimed to identify patterns of gray matter volume (GMV) alteration specific to and common among patients with RLS, migraine, and comorbid migraine and RLS."
6331309,gpt-4o,[StructuralMRI],[],None,True,[],"This study aimed to identify patterns of gray matter volume (GMV) alteration specific to and common among patients with RLS, migraine, and comorbid migraine and RLS."


,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
7426775,delavega-aliceoverlap,[StructuralMRI],[],None,False,None,NaN
7426775,gpt-4o-mini,[StructuralMRI],[],None,False,[],To predict chronological age from structural magnetic resonance imaging scans using a deep learning framework and to identify brain regions contributing to this prediction.
7426775,gpt-4o,[StructuralMRI],[],None,False,[],"The study aims to use a Deep Learning framework to predict chronological age from structural MRI scans and identify brain regions contributing to this prediction, focusing on brain aging."


,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
7582181,delavega-aliceoverlap,[StructuralMRI],[],None,False,None,NaN
7582181,gpt-4o-mini,[StructuralMRI],[],None,False,[],To investigate the relationship between social reward and threat expectancies and regional grey matter volumes (rGMV) in healthy adults using voxel-based morphometry (VBM).
7582181,gpt-4o,[StructuralMRI],[],None,False,[],To investigate the unique and overlapping regional grey matter volume (rGMV) correlates of inter-individual differences in social threat and reward expectancies using voxel-based morphometry (VBM).


,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
8564184,delavega-aliceoverlap,[StructuralMRI],[],None,False,None,NaN
8564184,gpt-4o-mini,[StructuralMRI],[],None,False,[],"To elucidate structural brain alterations in patients with chronic visceral pain, specifically comparing patients with ulcerative colitis and irritable bowel syndrome to healthy controls, and to assess associations with symptom severity and chronic stress."
8564184,gpt-4o,[StructuralMRI],[],None,False,[],Identify structural brain alterations in chronic visceral pain patients (UC and IBS) and investigate associations with visceral symptoms and chronic stress.


,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
9202476,delavega-aliceoverlap,"[fMRI-BOLD, DiffusionMRI]",[],None,True,None,NaN
9202476,gpt-4o-mini,"[fMRI-BOLD, DiffusionMRI]",[match-to-sample (MTS) task],None,False,[This computerized task assesses visual search and attention by presenting a complex figure and requiring subjects to match it with patterns shown in the periphery.],"The aim of the study was to evaluate correlation of sleep quality with visual attention and search, functional, and tracts properties of the DAN and VAN."
9202476,gpt-4o,"[fMRI-BOLD, DiffusionMRI, StructuralMRI]",[],None,True,[],"The aim of the study was to evaluate correlation of sleep quality with visual attention and search, functional, and tracts properties of the DAN and VAN."


On studies that previously either had outside the scanner tasks listed as in the scanner, or resting state incorrectly tagged as task, in many cases they are now appropriately extracted (with 1-2 exceptions)

In [182]:
# Diff on TaskName

has_taskname_df = combined_df[combined_df.index.isin(has_task_name)]

diff_taskname = set(all_scores_df[all_scores_df['TaskName'] < 0.8].index.tolist()).union(set(all_scores_df4[all_scores_df4['TaskName'] < 0.8].index.tolist()))

if diff_taskname:
    _display(has_taskname_df, diff_taskname)

,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
2241626,delavega-aliceoverlap,[fMRI-BOLD],"[mental calculation task, language comprehension task]",None,False,"[In the present research, our goal was to define a simple fMRI test, less than 5 minutes long, that could delineate, in a subject-specific manner, those cerebral circuits. A functional sequence was added to each functional imaging session performed in our lab (Figure ), taking advantage of the ...",NaN
2241626,gpt-4o-mini,[fMRI-BOLD],[Multi-functional localizer],None,False,"[A fast event-related fMRI protocol designed to capture individual cerebral correlates of visual, auditory, and sensorimotor processes, reading, language comprehension, and mental calculation.]","To define a simple fMRI test, less than 5 minutes long, that could delineate, in a subject-specific manner, cerebral circuits associated with various cognitive processes."
2241626,gpt-4o,[fMRI-BOLD],[Multi-functional localizer paradigm],None,False,"[The task investigates various cognitive processes including auditory and visual perception, motor actions, reading, language comprehension, and mental calculation.]","To design a simple and fast fMRI acquisition protocol that captures individual cerebral networks associated with auditory and visual perception, motor actions, reading, language comprehension, and mental calculation."


,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
4374765,delavega_nv,"[MRS, fMRI-BOLD]",[fear inducing paradigm],None,False,None,NaN
4374765,gpt-4o-mini,"[fMRI-BOLD, MRS]",[Fear Provocation Paradigm],None,False,[The task involved presenting still pictures of spiders and other control animals to elicit fear-related BOLD responses.],To investigate the relationship between GABA concentration and fear-related BOLD responses in the insula.
4374765,gpt-4o,"[fMRI-BOLD, Other]",[Fear provoking paradigm],None,False,"[Participants were shown images to provoke fear, including spiders, other animals, general negative, and neutral images from the IAPS.]",To investigate the relationship between GABA concentration and fear-related BOLD responses in the insula region.


,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
5324609,delavega-aliceoverlap,"[fMRI-BOLD, fMRI-BOLD]",[acquired equivalence task],None,False,"[participants learned a set of visual discriminations via trial‐and‐error (learning phase), and were subsequently tested on their ability to generalize what they had learned (generalization phase). Both learning and generalization occurred within a single scanning session and took place in the c...",NaN
5324609,gpt-4o-mini,[fMRI-BOLD],[Learning and Generalization Task],None,False,[Participants learned a set of visual discriminations via trial-and-error and were subsequently tested on their ability to generalize what they had learned.],To investigate the hippocampal role in gradually learning a set of spatial discriminations and subsequently generalizing them in an acquired equivalence task.
5324609,gpt-4o,[fMRI-BOLD],[Visual Discrimination and Generalization Task],None,False,[Participants learned a set of visual discriminations via trial-and-error and were subsequently tested on their ability to generalize what they had learned within a virtual reality environment.],Investigate the hippocampal role in gradually learning a set of spatial discriminations and subsequently generalizing them in an acquired equivalence task.


,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
6037859,delavega_nv,[fMRI-BOLD],"[synchrony judgment, temporal order judgment ]",None,False,"[The fMRI procedure was similar to the behavioral experiment, except that a reduced stimulus set was presented: −333, 0, PSS, and +333. The PSS values were obtained individually from the pre-fMRI experiment separately for SJ and TOJ. To be as accurate as possible the individual PSS conditions we...",NaN
6037859,gpt-4o-mini,[fMRI-BOLD],"[Synchrony Judgment (SJ), Temporal Order Judgment (TOJ)]",None,False,"[Participants decide whether audio and visual cues are in synch or out of synch., Participants decide which cue came first (or last).]",To investigate how behavioral differences between synchrony judgment (SJ) and temporal order judgment (TOJ) tasks are instantiated as neural differences using fMRI.
6037859,gpt-4o,[fMRI-BOLD],[Synchrony Judgments (SJ) and Temporal Order Judgments (TOJ) Tasks],None,False,[Tasks designed to assess how two different judgments—whether audiovisual cues are in synchrony (SJ) or which cue comes first (TOJ)—invoke different neural activations.],To investigate how behavioral differences between synchrony judgments (SJ) and temporal order judgments (TOJ) tasks are instantiated as neural differences using fMRI.


,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
6219793,delavega_nv,[fMRI-BOLD],[Social feedback paradigm],None,False,"[Following previous studies [ – ], negative social feedback was used to induce emotions for two reasons: (a) in daily life emotions are often caused by social stimuli [ , ] and (b) social feedback elicits emotional responses that are long enough to study emotion dynamics [ ]. The social feedback...",NaN
6219793,gpt-4o-mini,[fMRI-BOLD],[Self-Distancing vs. Self-Immersion Perspective Task],None,False,[Participants were asked to adopt a self-immersed or self-distanced perspective while reading and thinking about negative social feedback.],"To replicate and extend previous findings on the psychological and neural mechanisms underlying emotion explosiveness and accumulation, specifically examining the effects of self-distancing versus self-immersion on these emotional dynamics."
6219793,gpt-4o,[fMRI-BOLD],[Self-distancing perspective task],None,False,[Participants adopt either a self-immersed or self-distanced perspective while reading and thinking about negative social feedback to assess changes in negative affect.],"To replicate and extend previous findings on the psychological and neural mechanisms underlying emotion explosiveness and accumulation, examining whether adopting a self-distanced (vs. self-immersed) perspective modulates activity in brain regions associated with these emotional dynamics."


...

,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
6715348,delavega_nv,[fMRI-BOLD],[fear conditioning and extinction paradigm],None,False,None,NaN
6715348,gpt-4o-mini,[fMRI-BOLD],[Fear Conditioning Paradigm],None,False,[Participants underwent a differential fear conditioning paradigm during fMRI to study cerebellar activation related to predictions and prediction errors.],To address the contribution of the cerebellum in processing predictions and prediction errors during a fear conditioning paradigm.
6715348,gpt-4o,[fMRI-BOLD],[Differential Fear Conditioning],None,False,[Participants underwent a differential fear conditioning paradigm using visual stimuli as conditioned stimuli (CS) and an electric shock as an unconditioned stimulus (US).],None


,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
7018765,delavega-aliceoverlap,[fMRI-BOLD],[Testing Emotional Attunement and Mutuality (TEAM)],None,False,"[his task builds on past studies that have utilized error processing paradigms in the study of ER (e.g., ; ; ) by specifically examining dyadic error processing. Parent-adolescent dyads completed the TEAM task while simultaneously undergoing fMRI scanning. This task was developed to examine b...",NaN
7018765,gpt-4o-mini,[fMRI-BOLD],[TEAM task],None,False,"[The TEAM task is designed to probe emotional reactivity and regulation in a dyadic context, specifically examining dyadic error processing.]","The current study examined brain activation related to parenting and emotion regulation in parent-adolescent dyads during concurrent fMRI scanning with a novel task, the Testing Emotional Attunement and Mutuality (TEAM) task."
7018765,gpt-4o,[fMRI-BOLD],[Testing Emotional Attunement and Mutuality (TEAM) task],None,False,[The TEAM task is designed to probe emotional reactivity and regulation in a dyadic context by examining brain activation in both parents and adolescents when the other dyad member makes a costly error.],"The study examined brain activation related to parenting and emotion regulation (ER) in parent-adolescent dyads during concurrent fMRI scanning with the TEAM task, aiming to study dyadic error processing and how it relates to parenting practices."


,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
8857499,delavega_nv,[fMRI-BOLD],[giving task],None,False,"[adolescents divided either a small or large number of coins (giving magnitude manipulation) between themselves and either a friend or unfamiliar peer (target manipulation) in an audience or anonymous context (peer presence manipulation), #### Giving magnitude: giving small and large amounts \n ...",NaN
8857499,gpt-4o-mini,[fMRI-BOLD],[Giving Task],None,False,[Participants divided either a small or large number of coins between themselves and either a friend or unfamiliar peer in an audience or anonymous context.],"This pre-registered fMRI study investigated behavioral and neural correlates of adolescents' small versus large size giving in different social contexts related to target (i.e., giving to a friend or unfamiliar peer) and peer presence (i.e., anonymous versus audience giving)."
8857499,gpt-4o,[fMRI-BOLD],[Modified Dictator Game],None,False,"[Participants divided coins between themselves and another person in either a small or large giving condition, with target and peer presence manipulations.]",This pre-registered fMRI study investigated behavioral and neural correlates of adolescents small versus large size giving in different social contexts related to target and peer presence.


,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
9910278,delavega_nv,[fMRI-BOLD],"[emotion reappraisal, dietary self-control]",None,False,None,NaN
9910278,gpt-4o-mini,[fMRI-BOLD],"[Emotion Regulation Task, Dietary Self-Control Task]",None,False,"[Participants were presented with positive, negative, and neutral stimuli and were asked to either view or reappraise the content to regulate their feelings., Participants made choices about whether to eat presented food items based on their healthiness and tastiness ratings.]",To investigate behavioral and neural associations in self-regulation across emotion regulation and dietary choice tasks in human participants.
9910278,gpt-4o,[fMRI-BOLD],[Emotion Regulation Task],None,False,"[Investigates reappraisal of emotional scenes (positive, negative) to assess self-regulation.]",To investigate behavioral and neural associations in self-regulation across emotion reappraisal and dietary choice domains in human participants.


,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
11063816,delavega-other,"[fMRI-BOLD, fMRI-BOLD]",[six-alternative forced-choice cued-recognition task],None,False,"[On each trial, a cue and six potential targets were presented simultaneously on the screen. The cue was presented in the middle of the screen with the six possible targets; one target and five foils form the same category (e.g., if the target was hammer, the five foils would be other randomly s...",NaN
11063816,gpt-4o-mini,[fMRI-BOLD],[Cued-Recognition Task],None,False,[Participants performed a six-alternative forced-choice cued-recognition task to retrieve overlapping elements from learned events.],To assess whether episodic events continue to be reinstated holistically and whether hippocampal pattern completion continues to facilitate holistic reinstatement following a period of consolidation.
11063816,gpt-4o,[fMRI-BOLD],[Holistic Episodic Memory Retrieval Task],None,False,[A task to assess hippocampal pattern completion and its relationship to neocortical reinstatement across delays.],To assess whether episodic events continue to be reinstated holistically and whether hippocampal pattern completion continues to facilitate holistic reinstatement following a period of consolidation.


Many of the incorrect cases have a reasonable guess. Not clear GPT4o is better than mini though

In [183]:
# Doesn't have TaskName (according to annotations)

has_notaskname_df = combined_df[combined_df.index.isin(has_task_name) == False]

diff_notaskname = set(all_scores_df[all_scores_df['TaskName'] < 0.8].index.tolist()).union(set(all_scores_df4[all_scores_df4['TaskName'] < 0.8].index.tolist()))

if diff_notaskname:
    _display(has_notaskname_df, diff_notaskname)

,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
4488375,delavega_nv,[fMRI-BOLD],[n/a],None,False,[The task used in this experiment was programmed using PsychoPy [ ]. The task featured a 0-back and a 1-back condition that continuously switched from one another throughout the experimental session (see ). Our paradigm is broadly similar to the paradigm used by Smallwood and colleagues [ ] and...,NaN
4488375,gpt-4o-mini,[fMRI-BOLD],[Shape Location Decision Task],None,False,[Participants made decisions about the location of shapes presented on screen either based on the current trial (0-back) or the prior trial (1-back).],To explore the hypothesis that the role of the DMN in higher order cognition is to allow cognition to be shaped by information from stored representations rather than information in the immediate environment.
4488375,gpt-4o,[fMRI-BOLD],[0-back and 1-back task],None,False,[A novel paradigm was used to test the DMN function involving a 0-back task where shapes are identified based on current trial and 1-back task based on prior trial.],The study explores the hypothesis that the default mode network's role in higher order cognition is to allow cognition to be guided by information from stored representations rather than the immediate environment.


,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
4547715,delavega_nv,[fMRI-BOLD],[n/a],None,False,[A) The task was conducted in an event-related design. Six trials from either the liked or the uninteresting male acquaintance were combined in a set. Each set contained pseudo-randomized four target trials (shown in red) with two control trials (shown in blue). (B) Each trial was created by a 2...,NaN
4547715,gpt-4o-mini,[fMRI-BOLD],[Gift Attractiveness Judgment Task],None,False,"[Participants judged the attractiveness of gifts received from male acquaintances, focusing on the gift rather than the giver.]","To investigate how preferences for the giver and the type of gift interactively determine the reward value of a gift, specifically examining the neural processing for understanding a gift's social meaning."
4547715,gpt-4o,[fMRI-BOLD],[Gift Preference Task],None,False,[Participants judged the attractiveness of gifts from liked and uninteresting male acquaintances.],Investigating how preferences for a giver modulate the neural processing of a gift's social meaning.


,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
4914983,delavega_nv,[fMRI-BOLD],[n/a],None,False,None,NaN
4914983,gpt-4o-mini,[fMRI-BOLD],[Viewing Comedy Movie Clips],None,False,[Participants watched three comedy clips during fMRI scanning to study brain activity related to humor.],To investigate which brain areas respond similarly when watching relatively long (~5min) movie clips of comedy genre and how self-rated experienced humorousness explains similarities in hemodynamic activity in these areas.
4914983,gpt-4o,[fMRI-BOLD],[Viewing Comedy Clips],None,False,[Subjects watched comedy clips during fMRI to study neural processes underlying humor.],To investigate which brain areas respond similarly when watching comedy movie clips and to explore the relationship between experienced humorousness and hemodynamic activity.


,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
5607552,delavega-aliceoverlap,"[fMRI-BOLD, fMRI-BOLD]",[n/a],None,False,"[a degraded sentence is first experienced as very difficult to understand and, after a single presentation of its intact counterpart, the intelligibility of this same degraded sentence reaches near‐perfect level., functional run consisted of 9 D‐I‐D stimulus sets, each of which comprised three b...",NaN
5607552,gpt-4o-mini,[fMRI-BOLD],[DID stimulus set],None,False,"[The task involved presenting distorted sentences followed by their intact counterparts, and then the distorted sentences again to assess changes in intelligibility.]",We examined which brain areas are involved in the comprehension of acoustically distorted speech using an experimental paradigm where the same distorted sentence can be perceived at different levels of intelligibility.
5607552,gpt-4o,[fMRI-BOLD],[DID (Distorted-Intact-Distorted) stimulus set task],None,False,"[The task involves presenting acoustically distorted sentences to participants, followed by their intact versions, and then the distorted sentences again.]",Examine brain areas involved in the comprehension of acoustically distorted speech and how previous exposure to intact sentences enhances intelligibility using fMRI.


,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
5662713,delavega_nv,[fMRI-BOLD],[n/a],None,False,[Participants watched 24 short videos while in the MRI scanner. The videos lasted on average 38 seconds (range 29–48 s) and were taken from short films or videos posted on www.YouTube.com . All videos depicted a short narrative and were presented without sound. The stories centered around one...,NaN
5662713,gpt-4o-mini,[fMRI-BOLD],[Memory Encoding and Retrieval Task],None,False,[Participants watched and retrieved short videos while being scanned in an MRI scanner.],"To investigate memory reinstatement for detailed lifelike memories between encoding, immediate retrieval and one-week delayed retrieval."
5662713,gpt-4o,[fMRI-BOLD],[Memory Retrieval Task],None,False,"[Participants watched 24 short videos and retrieved the memories during two phases: immediate retrieval on the same day and delayed retrieval one week later, inside the MRI scanner.]","To investigate the memory reinstatement for detailed lifelike memories between encoding, immediate retrieval, and one-week delayed retrieval using functional MRI and representational similarity analyses (RSA)."


...

,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
8104963,delavega_nv,[fMRI-BOLD],[n/a],None,False,"[The main experimental session was organized in three blocks. In the first block, participants saw a fixation cross (1 s) followed by one video (3 s) and were asked to estimate the amount of pain experienced by the person in the video, by moving a randomly-presented rectangular cursor on a Visua...",NaN
8104963,gpt-4o-mini,[fMRI-BOLD],[Pain Assessment Task],None,False,[Participants appraised the pain of facial expression video-clips and subsequently were confronted with feedbacks from the person in pain and the average opinion of medical practitioners.],To test whether senior medical students differed from younger colleagues and lay controls in the way they assess people's pain and take into consideration their feedback.
8104963,gpt-4o,[fMRI-BOLD],[Pain facial expression appraisal task],None,False,[Participants evaluated the pain of facial expression video-clips and received feedback from the video's subject and medical practitioners.],"To investigate how senior medical students, younger colleagues, and lay controls differ in assessing people's pain and considering feedback, and how medical training affects sensitivity to pain faces and neural responses."


,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
8443248,delavega_nv,[fMRI-BOLD],[n/a],None,False,"[Participants were explicitly instructed to recreate the feelings of the demonstrators shown in the videos as vividly and intensely as possible. Based on the validation and pilot study, the painful expressions for the genuine and pretended conditions were matched. We also counterbalanced the...",NaN
8443248,gpt-4o-mini,[fMRI-BOLD],[Viewing Pain Expressions],None,False,[Participants watched video clips of individuals expressing genuine or pretended pain.],"To investigate the neural dynamics between the anterior insular cortex and right supramarginal gyrus in the context of genuine vs. pretended pain, focusing on affect sharing and self-other distinction."
8443248,gpt-4o,[fMRI-BOLD],[Pain Perception Task],None,False,[Participants watched video clips of people displaying either genuine or pretended facial expressions of pain.],"To investigate how the brain network involved in affect sharing and self-other distinction responds to genuine versus pretended pain, and to determine whether neural responses previously linked to affect sharing could result from the perception of salient affective displays."


,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
8927597,delavega_nv,[fMRI-BOLD],[n/a],None,False,None,NaN
8927597,gpt-4o-mini,[fMRI-BOLD],[2-back updating task (2Back)],None,False,[Participants were required to determine whether the current target was identical to the stimulus that appeared two trials before.],To examine the neural mechanisms associated with retrieval of weak representations using functional magnetic resonance imaging and their potential relationships with creativity task performance.
8927597,gpt-4o,[fMRI-BOLD],[2-back updating task (2Back) and classification task (Classification)],None,False,"[A task paradigm including a 2-back updating task, which appeared after a classification task, to differentiate the retrieval processes for previously ignored vs. attended stimuli.]",To examine the neural mechanisms associated with retrieval of weak representations using functional magnetic resonance imaging and their potential relationships with creativity task performance.


,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
8975992,delavega_nv,[fMRI-BOLD],[n/a],None,False,None,NaN
8975992,gpt-4o-mini,[fMRI-BOLD],[Naturalistic Stimuli Presentation],None,False,[Participants watched a Hollywood movie and listened to its audio-description to investigate spatial information processing.],"To investigate the perception of scene-related, spatial information embedded in two naturalistic stimuli (a movie and its audio-description) and to model hemodynamic activity based on annotations of selected stimulus features."
8975992,gpt-4o,[fMRI-BOLD],[Movie Watching Task],None,False,[Participants watch a Hollywood movie (Forrest Gump) while hemodynamic activity is monitored.],"Investigating the perception of scene-related, spatial information embedded in naturalistic stimuli, specifically a Hollywood movie and its audio-description, to understand activation in the parahippocampal place area (PPA)."


,annotator_name,Modality,TaskName,Exclude,HasRestingState,TaskDescription,StudyObjective
10028637,delavega_nv,[fMRI-BOLD],[n/a],None,True,None,NaN
10028637,gpt-4o-mini,[fMRI-BOLD],[Auditory Target Detection Task],None,False,[A computerized auditory target detection task aimed to assess individual responsiveness differences during moderate anaesthesia.],"To investigate whether variability or impairments in functional connectivity within and between the dorsal attention network (DAN), executive control network (ECN), and default mode network (DMN) underlie individual differences in responsiveness during propofol anaesthesia."
10028637,gpt-4o,"[fMRI-BOLD, StructuralMRI]",[Narrative Understanding Task],None,True,[Participants listened to a plot-driven auditory narrative while undergoing fMRI scans to investigate perceptual or high-level attention processes under anaesthesia.],"To investigate whether variability or impairments in functional connectivity (FC) within and between the dorsal attention network (DAN), executive control network (ECN), and default mode network (DMN) underlie individual differences in responsiveness during propofol anaesthesia."


Often times a reasonable task name is suggested, but sometimes a behavioral tasks is confused for scanner task

In [184]:
##  Focus on TaskDescription

has_notaskname_df = combined_df[combined_df.index.isin(has_task_name) == False]

In [265]:
import json 
def _loop_print_row(df, fields=['TaskName', 'TaskDescription']):
    for _, row in df.iterrows():
        row[fields]
        print(json.dumps(row[fields].to_dict(), indent=4))

def _print_extracted_data(df):
    for _, row in df.iterrows():        
        # Print Behavioral Tasks
        for task_type in ['BehavioralTasks', 'fMRITasks']:
            _val = row[task_type]
            if _val and isinstance(_val, list):
                print(f'{task_type}:')
                for task in _val:
                    print(json.dumps(task, indent=4))
        print('')

def clean_print_by_pmcid(df):
    for pmcid, _df in df.groupby('pmcid'):
        print('PMCID:', pmcid)

        for annotator_name, _extract in _df.groupby('annotator_name'):
            print(f'--- {annotator_name} ---')
            if 'delavega' in annotator_name:
                _loop_print_row(_extract)
            else:
                print('\tStudyObjectve:', _extract['StudyObjective'].iloc[0])
                _print_extracted_data(_extract)
            print('')


        print('=============================================')

In [272]:
# Modify the following so instead of printing it appends to a list that can be used to generate a report
import json 
def _loop_print_row(df, fields=['TaskName', 'TaskDescription']):
    for _, row in df.iterrows():
        row[fields]
        print(json.dumps(row[fields].to_dict(), indent=4))

def _print_extracted_data(df):
    for _, row in df.iterrows():        
        # Print Behavioral Tasks
        for task_type in ['BehavioralTasks', 'fMRITasks']:
            _val = row[task_type]
            if _val and isinstance(_val, list):
                print(f'{task_type}:')
                for task in _val:
                    print(json.dumps(task, indent=4))
        print('')

def clean_print_by_pmcid(df):
    for pmcid, _df in df.groupby('pmcid'):
        print('PMCID:', pmcid)

        for annotator_name, _extract in _df.groupby('annotator_name'):
            print(f'--- {annotator_name} ---')
            if 'delavega' in annotator_name:
                _loop_print_row(_extract)
            else:
                print('\tStudyObjectve:', _extract['StudyObjective'].iloc[0])
                _print_extracted_data(_extract)
            print('')


        print('=============================================')

In [273]:
clean_print_by_pmcid(has_taskname_df)

PMCID: 2241626
--- delavega-aliceoverlap ---
{
    "TaskName": [
        "mental calculation task",
        "language comprehension task"
    ],
    "TaskDescription": [
        "In the present research, our goal was to define a simple fMRI test, less than 5 minutes long, that could delineate, in a subject-specific manner, those cerebral circuits. A functional sequence was added to each functional imaging session performed in our lab (Figure  ), taking advantage of the continuous flow of volunteers recruited for various protocols. Because we wanted to capture the maximal amount of functional information in the minimum amount of time, we designed the sequence according the following challenging constraints: \n\n\u25aa the sequence had to be short, so as to disrupt as little as possible the main protocol. We choose 5 minutes for performing 100 trials. \n\n\u25aa we aimed to obtain for each subject a description of different levels of functional architecture, from sensori-motor areas (per